In [121]:
import pandas as pd
import numpy as np

import pymc3 as pm
import arviz as az
import theano.tensor as tt

import matplotlib.pyplot as plt
import seaborn as sns
import pygal

from sklearn.metrics import mean_absolute_percentage_error as mape

import warnings
warnings.filterwarnings('ignore')

In [4]:
from helper import *

In [28]:
mediaFile = "proactiv_usa_march23.csv"
dfMedia = pd.read_csv(mediaFile).drop(columns=["holiday"])
dfMedia.head()

,Day,Amazon_Prospecting_MediaCost,BingSearch_PaidSearchBrand_MediaCost,BingSearch_PaidSearchNonBrand_MediaCost,BingSearch_ShoppingAdsBrandNonBrand_MediaCost,CTV_Awareness_MediaCost,Criteo_Prospecting_MediaCost,Criteo_Retargeting_MediaCost,DV360_Prospecting_MediaCost,DV360_Retargeting_MediaCost,...,Snapchat_ConversionsYoungAdults_Impressions,Snapchat_VideoViewsTeens_Impressions,Snapchat_VideoViewsYoungAdults_Impressions,TheTradeDesk_Prospecting_Impressions,TheTradeDesk_Retargeting_Impressions,Tinder_Prospecting_Impressions,Twitch_Prospecting_Impressions,YouTube_Prospecting_Impressions,YouTube_Retargeting_Impressions,Orders
0,2021-03-01,0.0,399.43,52.02,170.28,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,87163.0,24346.0,780
1,2021-03-02,0.0,356.26,51.91,149.57,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,83037.0,30554.0,769
2,2021-03-03,0.0,418.41,37.60,185.69,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,93431.0,31253.0,797
3,2021-03-04,0.0,363.64,35.39,135.36,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,84057.0,35405.0,741
4,2021-03-05,0.0,403.17,42.96,127.99,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60483.0,33950.0,706


In [29]:
dfMedia = dfMedia[get_media_vars(dfMedia) + ["Orders"]]

In [30]:
dfMediaCombined = pd.DataFrame()
fringe = set([])
cols = sorted(dfMedia.columns.values)
for col in cols:
    short = shorten_f_name(col)
    if short in fringe:
        dfMediaCombined[f"{short}_media_cost".lower()] += dfMedia[col]
    else:
        dfMediaCombined[f"{short}_media_cost".lower()] = dfMedia[col]
    fringe.add(col)

In [31]:
dfMediaCombined["orders"] = dfMedia.Orders

In [40]:
dfMediaCombined.isna().sum()

amazon_media_cost          0
bingsearch_media_cost      0
ctv_media_cost             0
criteo_media_cost          0
dv360_media_cost           0
facebook_media_cost        0
googlesearch_media_cost    0
influential_media_cost     0
lineartv_media_cost        0
orders_media_cost          0
pinterest_media_cost       0
radio_media_cost           0
snapchat_media_cost        0
thetradedesk_media_cost    0
tinder_media_cost          0
twitch_media_cost          0
youtube_media_cost         0
orders                     0
dtype: int64

checking model assumptions

In [43]:
high = 0
threshold = 0.3
n = dfMediaCombined.shape[1]
for i in range(0, n):
    for j in range(i+1, n):
        if i != j:
            col1, col2 = dfMediaCombined.iloc[:, i], dfMediaCombined.iloc[:, j]
            r = np.corrcoef(col1, col2)[0][1]
            if abs(r) >= threshold:
                high += 1      
print(f"{high} pairs of features out of {int(n*(n+1) / 2)} have correlation exceeding threshold={threshold}")

22 pairs of features out of 171 have correlation exceeding threshold=0.3


In [56]:
too_low = 0
percent_variance = 0.2 # threshold=20%
n = dfMediaCombined.shape[1]
features = []
for i in range(0, n):
    col = dfMediaCombined.iloc[:, i]
    stddev = np.std(col)
    if stddev < (percent_variance * np.mean(col)):
        too_low += 1
        features.append(dfMediaCombined.columns.values[i])
print(f"{too_low} features have insufficient variability")
features

0 features have insufficient variability


[]

## saturation and carryover

In [ ]:
def saturate(x, a):
    """
        arbitrary saturation curve, parameters of this function must define saturation curve
    """
    return 1 - tt.exp(-a*x)

def carryover(x, strength, length=21):
    """
        same function as specified in google whitepaper
        usually use poission random variable for length
    """
    w = tt.as_tensor_variable(
        [tt.power(strength, i) for i in range(length)]
    )
    
    x_lags = tt.stack(
        [tt.concatenate([
            tt.zeros(i),
            x[:x.shape[0]-i]
        ]) for i in range(length)]
    )
    
    return tt.dot(w, x_lags)

def plot_saturation():
    """
        plots saturation curve with preset parameters
    """
    plt.style.use('ggplot')
    plt.figure(figsize=(6, 4))
    a = 5e-5
    x = np.arange(0, 100000, 0.1)
    y = 1 - np.exp(-a*x)
    plt.plot(x, y, color='green')
    plt.ylabel("Channel Saturation")
    plt.title(f"Saturation Curve");
    
def mape(mmm):
    """
        mmm: pm.Model() after it has been trained
    """
    
    with mmm:
        posterior = pm.sample_posterior_predictive(trace)
    means = posterior['sales'].mean(0)
    stds = posterior['sales'].std(0)
    
    plt.figure(figsize=(20, 8))
    plt.plot(y.values, linewidth=2, c='r', label='Observations')
    plt.plot(means, linewidth=1, c='b', label='Mean prediction')
    plt.fill_between(np.arange(len(y)), means - 2*stds, means + 2*stds, alpha=0.33)
    plt.legend()

In [ ]:
class BayesianMixModel:
    def __init__(self, data, target):
        """
            data: DataFrame containing both X and y
            target: (str) column in data that is the response variable
        """
        self.data = data
        self.target = target
        
        self.X = data.drop(columns=[target])
        self.y = data[target]
        
        self.train_model()
    
    def train_model(self):
        """
            called immediately upon initialization of BayesianMixModel instance
            trains model
        """
        with pm.Model() as mmm:
            channel_contributions = []

            for channel in self.X.columns:
                coef = pm.Exponential(f'coef_{channel}', lam=0.0001)
                sat = pm.Exponential(f'sat_{channel}', lam=1)
                car = pm.Beta(f'car_{channel}', alpha=2, beta=2)

                channel_data = self.X[channel].values
                channel_contribution = pm.Deterministic(
                    f'contribution_{channel}',
                    coef * saturate(
                        carryover(
                            channel_data,
                            car
                        ),
                        sat
                    )
                )

                channel_contributions.append(channel_contribution)

            base = pm.Exponential('base', lam=0.0001)
            noise = pm.Exponential('noise', lam=0.0001)

            sales = pm.Normal(
                'sales',
                mu=sum(channel_contributions) + base,
                sigma=noise,
                observed=y
            )

            trace = pm.sample(return_inferencedata=True, tune=3000)
        
        self.mmm = mmm
        self.trace = trace
            
    def actualVsFittedGraph(self):
        """
            plots actual vs fitted model
        """
        with self.mmm:
            posterior = pm.sample_posterior_predictive(self.trace)
        
        means = posterior['sales'].mean(0)
        stds = posterior['sales'].std(0)

        plt.figure(figsize=(20, 8))
        plt.plot(self.y.values, linewidth=2, c='r', label='Observations')
        plt.plot(means, linewidth=1, c='b', label='Mean prediction')
        plt.fill_between(np.arange(len(y)), means - 2*stds, means + 2*stds, alpha=0.33)
        plt.legend()
    
    def mape(self):
        """
            returns mape (on entire dataset)
        """
        with self.mmm:
            posterior = pm.sample_posterior_predictive(self.trace)
        
        means = posterior['sales'].mean(0)
        stds = posterior['sales'].std(0)
        
        return mape(y_true=self.y.values, y_pred=means)
    
    
        
        
        